In [2]:
import pandas as pd
import numpy as np
import ast
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob

Abrimos el archivo y lo pasamos a un DataFrame

In [4]:
df_reviews = pd.read_parquet(r'D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\australian_user_reviews.parquet')
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Desanidamos la columna 'reviews'

In [5]:
# utilizamos la función explode para la columna y poder desanidar los datos
desanidado = df_reviews.explode('reviews')
# normalizamosa los datos 
normalizar = pd.json_normalize(desanidado['reviews'].dropna())

In [6]:
normalizar.reset_index(inplace=True)
desanidado.reset_index(inplace=True)

In [7]:
# concatenamos los datos originales y eliminamos la columna original 'reviews' andidada
df_reviews = pd.concat([desanidado,normalizar], axis=1)
df_reviews = df_reviews.drop(columns=['reviews'])
df_reviews.head(1)

,index,user_id,user_url,index,funny,helpful,item_id,last_edited,posted,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...


In [12]:
#eliminamos la columna index ya que es innecesaria
df_reviews.drop(columns='index', inplace=True)

In [13]:
df_reviews.head(1)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...


## eliminamos duplicados 

In [14]:
df_reviews = df_reviews.drop_duplicates(keep='first')

In [15]:
df_reviews = df_reviews.dropna().reset_index(drop=True)

In [17]:
df_reviews.head(1)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...


## Transformamos los datos de tipo fecha en 'posted'

In [18]:
# extraemos la columna 'posted' 
df_reviews['posted'] = df_reviews['posted'].str.extract(r'Posted([\w\s\d,]+)')
df_reviews.head(2)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"November 5, 2011",True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,22200,,"July 15, 2011",True,It's unique and worth a playthrough.


In [20]:
#reemplazamos la palabra Posted por un espacio vacío
df_reviews['posted'] = df_reviews['posted'].replace({'Posted': ''}, regex= True)
df_reviews.head(1)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"November 5, 2011",True,Simple yet with great replayability. In my opi...


In [21]:
#transformamos la columna a tipo de dato datetime
df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], errors='coerce')
df_reviews.head(2)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,2011-11-05,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,22200,,2011-07-15,True,It's unique and worth a playthrough.


### Guardamos el DF en un archivo tipo parquet

In [22]:
df_reviews.to_csv('Archivos/reviews.csv', index=False, encoding='utf-8')

In [25]:
#tansformamos el csv a parquet
df_reviews = pd.read_csv(r'D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\reviews.csv')

tabla = pa.Table.from_pandas(df_reviews)
pq.write_table(tabla,'Archivos/reviews.parquet')
